# From tsv to JSON for original and transliterated files

This notebook converts the sanai lyrics tsv files to JSON files, for both tsvs in transliterated and original script.

## Prerequisites
1. Input Format:
    The expected formats of the tsv files is the that output from the 1) docx_to_tsv notebook.
2. Paths: 
    original script tsv files should be placed in data/original/tsv, and transliterated tsvs should be placed in data/transliterated/tsv. 
    

## Output JSON Format

don't forget to put the mag-houl or missing type.
when it is the first row of the sanaa, we can set the 'identifier' as the first data cell section

first item is the sanaa type, the name is slightly misleading, it refers to the type of the text, whether it is a sanaa, a mawwal, or an inshad. 

exception to the format dotting rule is that sometimes the length of the array after splitting based on dots is 1, and therefore. 

stype could be a sanaa, or tawshiyat al mizan. 

For tawshiyat al mizan, the identifier could be empty (in cases where there are no lyrics indicated for the tawshiya) or could have the first line section (if there are lyrics).

The data for each line in the sanaa is an array of at least 2 entries. First one is either a numeric entry representing the cycles in which that line is performed. If the numeric entry is not present (due to missing data at the time of the lyrics corpus creaton), then at least an empty string will be present in place of the first entry.
After that entry, one or more lyrics line sections are present, each as its own string entry in the array corresponding to a line.

The word sanaa is used (in variable names etc), but it should be noted that technically speaking, we can be currently processing a sanaa, a tawshiya, an inshad, or a mawwal. So the use of recording_sanas as a variable should not confuse the user. At the end this array will have all the elements that appeared in the lyrics of the recording.

The 'section' entry in the dictionary corresponding to a sanaa/inshad/mawwal takes the format of [ [ [line], [line] ] ]]. It's an array of sections, where each section is an array of lines. given that each line is in itself an array, this makes the section entry a 3D array.

Be aware that sometimes spacing between words in arabic could cause substring mismatches and eventually cause bugs. For example, (tawshiyat al mizan) is without spaces. When creating the lyrics files, check this notebook to see which keywords are expected in which format to make sure the script works correctly.

Note that the metadata.tsv of the github repository is different than the metadata.tsv of the lyrics corpus, because while the former should have metadata on all the recordings that have been processed by the MTG (whether they have lyrics or not) the metadata.tsv of the lyrics corpus are only those that correspond to lyrics files. 

1. Lyrics

(a) Hijaz Mshrqi, Darj: b018d0fd-9955-47bb-9ef2-d291f500eebe

i. 2 Sanaas are missing

ii. One sanaa was retrieved from the book under Qaim wa Nisf (specifically awqadta fih)

iii. 2 Sanaas are from Bassit (ya qawmi and fi almalih)

iv. There are 4 unknown minutes between the end of unzur ila (36:56) and the beginning of fil malih. The score indicates it is a Sanaa called ya qawmi but that is not the case

v. The last 4 sanaas are from btayhi (koll yawm, saqeit ka's elhawa, min 3ando khal, bimuhjati)

(b) Hijaz Mshrqi, 64eb6302-34ec-4434-a004-4e7e503c3e6c

i. Lyrics file could almost be a subset of a1782 (so were derived from it), but I was unable to detect the last sanaa

(c) Quddam Istihlal: 2d2683c4-4b3e-4430-a254-c828427bdcc7

i. Only the mawwal at the beginning was inserted

(d) Rasd: bb7adfee-f63d-4004-a457-b31ae784a0bd

i. missing inshad in lyrics file

(e) 0fb722f6-35a3-436b-9316-4c2de2b86cc7: first sanaa unknown. 

(f) c1e0311e-ee21-470c-b6f9-815d892cf6c9: needs completion

In [7]:
import sys, glob, json, shutil, codecs, platform
import pandas as pd
import ArabicTransliterator

from IPython.display import display, HTML

## JSON Conversion

s is either a file stream or a string stream. 
v indicates whether we are using the original or transliterated version of the lyrics, where 'o' refers to the original and 't' refers to the transliterated. The need to have the transliteration status as an input is because some provisions are needed in order to handle some exceptions/missing items in the corpus, and in those cases words from the files need to be checked.

In [8]:
def strip_digits(s):
    return ''.join([i for i in s if not i.isdigit()])

def tsv_to_json(s, v):
    recording_sanas = []
    keywords = {'o' : {'type1' :['توشية الميزان', 'توشيةالميزان', 'الصنعة', 'إنشاد', 'موال', 'صنعة', 'توشيةالنوبة'], 'type2' : ["توشية"] } , 
                't': {'type1' : ["tawshīt al-mīzān", "tawshīt al-mīzān", "al-ṣan‘ah", "inshād", "mawwāl", "ṣan‘ah", "tawshīt al-nūbah"], 'type2' : ["tawshīya"] } }
    missing = {'o' : 'مجهول', 't' : 'unidentified'}
    known_ptypes = {'t': ['zajal','qaṣīdah', 'tawshīḥ', 'birūlatin', 'birūālatin', 'tawshīḥun', 'qaṣīdatun'], 'o' : ['زجل', 'قصيدة', 'توشيح', 'برولة']}
    
    poem_types, sana_types = {}, {}
    for line in s:
        data = line.strip('\r\n').rstrip('\t').split("\t")
        if len(data) == 1:       #if no tabs found
            if len(data[0]) > 0: #and non empty line
                title = data[0].strip('\r\n').split(".")
                stype, ptype = "", ""
                stype = strip_digits(title[0]).strip()  
                
                if stype not in keywords[v]['type1']:
                    stype = missing[v]     
                sana_types.setdefault(stype, 0)
                sana_types[stype] += 1
            
                ptype_matches = [p for p in known_ptypes[v] if p in line] #if more than 1 match, we use first.
            
                if len(ptype_matches) > 0:
                    ptype = ptype_matches[0]
                else:
                    ptype = missing[v]
                poem_types.setdefault(ptype, 0)
                poem_types[ptype] += 1
                
                recording_sanas.append({"title": data[0].strip(), "sections": [[]], "type": stype, "identifier": "", "poem" : ptype})
                
            else: 
                recording_sanas[-1]['sections'].append([])
              
        elif len(data) == 2 and (data[1] == keywords[v]['type1'][0] or data[1] == keywords[v]['type1'][1]): #"tawshīt al-mīzān" or its arabic script equiv.
            stype = data[1]
            recording_sanas.append({"title": data[1].strip(), "sections": [[]], "type": stype, "identifier": "", "poem" : ""})
    
        elif len(data) >= 2: #line in a sanaa body
            if(len(recording_sanas[-1]['sections'][0])) == 0: #if this is the first row in the sanaa
                recording_sanas[-1]['identifier'] = data[1] 
            recording_sanas[-1]['sections'][-1].append(data)
             
    #some post-processing
    for i in range(len(recording_sanas)): 
        if len(recording_sanas[i]['sections'][-1]) == 0: #remove empty sections in a recording if found.. 
            recording_sanas[i]['sections'] = recording_sanas[i]['sections'][:-1]
        if recording_sanas[i]['identifier'] == keywords[v]['type2'][0]:  #if the identifier is a tawshiya, take the following line section
            if len(recording_sanas[i]['sections'][0][0]) > 1: 
                recording_sanas[i]['identifier'] = recording_sanas[i]['sections'][0][0][1]
    
    return (recording_sanas, sana_types, poem_types)

This cell concerns the loading of files, and the actual tsv to json conversion, both when in original and transliterated formats

In [9]:
original_files = sorted(glob.glob("data/original/tsv/*.tsv"))
transliterated_files = sorted(glob.glob("data/transliterated/tsv/*.tsv"))

processed_original_files = []
processed_transliterated_files = []

for filename in original_files:
    mbid = filename[filename.rfind("/")+1:filename.rfind(".")]
    f = codecs.open(filename, "r", "utf-8")
    (recording_sanas, sana_types, poem_types) = tsv_to_json(f, 'o')
    f.close()
    json.dump(recording_sanas, codecs.open("data/original/json/%s.json" % mbid, "w+", "utf-8"))
    processed_original_files.append(filename)
    display(filename)
    
    for stype, freq in sorted(sana_types.items(), key=lambda x: x[1], reverse=True):
        print(stype)
        print(freq)

    for ptype, freq in sorted(poem_types.items(), key=lambda x: x[1], reverse=True):
        print(ptype)
        print(freq)
        
for filename in transliterated_files:
    mbid = filename[filename.rfind("/")+1:filename.rfind(".")]
    f = codecs.open(filename, "r", "utf-8")
    (recording_sanas, sana_types, poem_types) = tsv_to_json(f, 't')
    f.close()
    json.dump(recording_sanas, codecs.open("data/transliterated/json/%s.json" % mbid, "w+", "utf-8"))
    processed_transliterated_files.append(filename)
    display(filename)
    
    for stype, freq in sorted(sana_types.items(), key=lambda x: x[1], reverse=True):
        print(stype)
        print(freq)

    for ptype, freq in sorted(poem_types.items(), key=lambda x: x[1], reverse=True):
        print(ptype)
        print(freq)

'data/original/tsv/023b4a37-1ab4-4593-b03a-850ee0db8350.tsv'

الصنعة
16
زجل
8
توشيح
6
قصيدة
2


'data/original/tsv/02b76946-deec-48cf-9007-770c37b6c3cc.tsv'

الصنعة
10
زجل
6
توشيح
3
قصيدة
1


'data/original/tsv/03e555c6-a480-4e11-aef7-231891c4f4e6.tsv'

...

al-ṣan‘ah
10
inshād
1
zajal
8
unidentified
2
tawshīḥ
1


'data/transliterated/tsv/f8b75f4a-1524-43cb-ad45-622ac277d96e.tsv'

al-ṣan‘ah
6
unidentified
1
zajal
2
qaṣīdah
2
tawshīḥ
2
unidentified
1


'data/transliterated/tsv/fb387964-4764-4900-91b5-f2ea7e10b786.tsv'

al-ṣan‘ah
9
zajal
8
qaṣīdah
1
